# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps.datasets
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Loan CSV file generated from WeatherPy Folder
output_data_to_load = "../output_data/cities.csv"
city_df = pd.read_csv(output_data_to_load)
city_df


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Tiksi,71.6872,128.8694,-10.71,100,100,6.58,RU,1643674237
1,Puerto Carreño,6.1890,-67.4859,86.99,33,65,6.38,CO,1643674644
2,Richards Bay,-28.7830,32.0377,74.35,82,61,9.89,ZA,1643674645
3,Rikitea,-23.1203,-134.9692,78.01,67,66,4.72,PF,1643674646
4,Port Alfred,-33.5906,26.8910,74.95,96,100,5.82,ZA,1643674647
...,...,...,...,...,...,...,...,...,...
535,Moindou,-21.6924,165.6770,78.75,67,96,5.75,NC,1643675086
536,San Miguel de Cozumel,20.5083,-86.9458,70.95,77,20,3.00,MX,1643675048
537,Bonavista,48.6499,-53.1147,32.11,70,63,15.37,CA,1643675088
538,Ferkessédougou,9.6000,-5.2000,74.23,15,32,4.94,CI,1643675088


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [14]:
# Configure gmaps

gmaps.configure(api_key=g_key)

# Store latitude and longitude in locations
locations = city_df[["Lat", "Lng"]]

# Store Humidity in humidity
humidity =city_df["Humidity"]


In [20]:

# Plot Heatmap   

locations = city_df[["Lat", "Lng"]]
humidity = city_df["Humidity"]

fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)
max_intensity = np.max(humidity)

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False)
heat_layer.max_intensity = 100
heat_layer.point_radius = 3


fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [21]:
clean_df = city_df[(city_df["Max Temp"] >= 75) & (city_df["Max Temp"] <= 90)]
clean_df = clean_df[clean_df["Wind Speed"] <= 10]
clean_df = clean_df[clean_df["Cloudiness"] <= 10]
clean_df = clean_df[clean_df["Humidity"] <= 70]
hotel_df = clean_df
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
18,Kapaa,22.0752,-159.3190,78.78,70,0,3.44,US,1643674658
36,Port Macquarie,-31.4333,152.9167,85.89,66,8,3.29,AU,1643674673
216,‘Āhuimanu,21.4447,-157.8378,80.31,63,0,3.00,US,1643674823
275,Concordia,-31.3930,-58.0209,78.78,59,0,7.00,AR,1643674803
333,Cabo San Lucas,22.8909,-109.9124,75.61,61,1,3.00,MX,1643674699
399,Las Varas,21.1667,-105.1667,85.48,58,0,4.61,MX,1643674972
422,Acajutla,13.5928,-89.8275,78.80,64,3,8.12,SV,1643674991
467,Makakilo City,21.3469,-158.0858,79.12,58,0,8.05,US,1643674801
528,Neuquén,-38.9516,-68.0591,78.69,25,0,5.75,AR,1643675081


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [22]:
hotel_df['Hotel Name'] = ""
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
18,Kapaa,22.0752,-159.3190,78.78,70,0,3.44,US,1643674658,
36,Port Macquarie,-31.4333,152.9167,85.89,66,8,3.29,AU,1643674673,
216,‘Āhuimanu,21.4447,-157.8378,80.31,63,0,3.00,US,1643674823,
275,Concordia,-31.3930,-58.0209,78.78,59,0,7.00,AR,1643674803,
333,Cabo San Lucas,22.8909,-109.9124,75.61,61,1,3.00,MX,1643674699,
399,Las Varas,21.1667,-105.1667,85.48,58,0,4.61,MX,1643674972,
422,Acajutla,13.5928,-89.8275,78.80,64,3,8.12,SV,1643674991,
467,Makakilo City,21.3469,-158.0858,79.12,58,0,8.05,US,1643674801,
528,Neuquén,-38.9516,-68.0591,78.69,25,0,5.75,AR,1643675081,


In [23]:
for index, row in hotel_df.iterrows():
    try:
        
        base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

        params = {
        "keyword": "hotel",
        "radius": 5000,
        "key": g_key,
        }


        lat = row['Lat']
        lng = row['Lng']

        params['location'] = f"{lat}, {lng}"

        hotel_data = requests.get(base_url, params=params).json()
        
        hotel_df.loc[index, "Hotel Name"] = hotel_data["results"][0]["name"]
        
    except IndexError:
        
        hotel_df.loc[index, "Hotel Name"] = "NaN"
        
hotel_df        

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
18,Kapaa,22.0752,-159.3190,78.78,70,0,3.44,US,1643674658,Hotel Coral Reef
36,Port Macquarie,-31.4333,152.9167,85.89,66,8,3.29,AU,1643674673,Mantra The Observatory Port Macquarie
216,‘Āhuimanu,21.4447,-157.8378,80.31,63,0,3.00,US,1643674823,NaN
275,Concordia,-31.3930,-58.0209,78.78,59,0,7.00,AR,1643674803,Cristobal Hotel Boutique
333,Cabo San Lucas,22.8909,-109.9124,75.61,61,1,3.00,MX,1643674699,Solmar Resort
399,Las Varas,21.1667,-105.1667,85.48,58,0,4.61,MX,1643674972,NaN
422,Acajutla,13.5928,-89.8275,78.80,64,3,8.12,SV,1643674991,Villavela Hotel Boutique
467,Makakilo City,21.3469,-158.0858,79.12,58,0,8.05,US,1643674801,Hampton Inn & Suites Oahu/Kapolei
528,Neuquén,-38.9516,-68.0591,78.69,25,0,5.75,AR,1643675081,Neuquen Tower Hotel


In [24]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [25]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)
fig

# Display figure


Figure(layout=FigureLayout(height='420px'))